[View in Colaboratory](https://colab.research.google.com/github/IAmSuyogJadhav/Emotion-Recognition/blob/master/train.ipynb)

In [1]:
# For Google Colab
# Uncomment following code for mounting Google Drive

# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
# !mkdir -p drive
# !google-drive-ocamlfuse drive
# %cd drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
/content/drive


In [0]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(777)

In [3]:
# For Google Colab
# %cd ML/Emotion-Recognition

/content/drive/ML/Emotion-Recognition


In [0]:
#@title Specify the image size here
img_size = 25 #@param {type:"slider", min:10, max:50, step:1}


In [0]:
# UNCOMMENT FOLLOWING CODE TO GENERATE TRAIN DATA OF REQUIRED SIZE


# import os
# train_list = os.listdir("./images_new")
# sample_img = plt.imread('./images_new/' + train_list[2])
# print(sample_img.shape)
# plt.imshow(sample_img)

# img_size = 25
# train = np.zeros((len(train_list), img_size, img_size))

# print(train.shape)
# train_labels = np.zeros(len(train_list))

# print(train_labels.shape)

# labels = pd.read_csv('labels.csv')
# to_int = {
#     'anger': 0,
#     'contempt': 1,
#     'disgust': 2,
#     'fear': 3,
#     'happiness': 4,
#     'neutral': 5,
#     'sad': 6,
#     'sadness': 7,
#     'surprise':8   
# }

# from skimage import io
# from skimage.transform import resize
# np.random.shuffle(train_list)
# # j=0
# j = 7588
# # for i in range(len(train_list)):
# for i in range(7610, len(train_list)):
#     path = train_list[i]
#     print('%d\t' % (i) if i%100 else '\r', end=' ')
#     img = resize(io.imread('./images_new/' + path), (img_size, img_size))
#     try:    
#         label = labels[labels.image == path].emotion.values[0]
#         train_labels[j] = to_int[label]
#         train[j] = img
#         j += 1
#     except:
#         print("Failed: i=%d"%i)
#         pass
      
# train1 = train[:j]
# train_labels1 = train_labels[:j]
# np.save('train_%dx%d' % (img_size, img_size), train1)
# np.save('train_labels_%dx%d' % (img_size, img_size, train_labels1)

In [0]:
#@title Choose Image Size to load { form-width: "10%", display-mode: "both" }
size = "25x25" #@param ["25x25", "50x50"]


train = np.load('train_{}.npy'.format(size))
train = train.reshape(tuple(list(train.shape) + [1])) # reshaping to adjust in conv layer.
train_labels = np.load('train_labels_{}.npy'.format(size))

In [0]:
from keras import Sequential
from keras import layers as ll
from keras.utils import to_categorical
from keras import regularizers
import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Data Augmentation

datagen = ImageDataGenerator(
          rotation_range=15,
          width_shift_range=0.2,
          height_shift_range=0.2,
          zoom_range = 0.1,
          horizontal_flip=True,     
)

i = 0
for batch in datagen.flow(x=train, y=train_labels, batch_size=2, 
                          save_to_dir='augmented', save_format='jpeg'):
  i+=1
  if i > 20:
    break

In [0]:
#@title Adjust hyper parameters { display-mode: "form" }
alpha = 0.3 #@param {type:"number"}
pool_size = 2 #@param {type:"slider", min:1, max:5, step:1}
dropout = 0.2 #@param {type:"slider", min:0, max:1, step:0.05}
l2_regularization = 0.2 #@param {type:"number"}


In [0]:
img_size = train.shape[1]

model = Sequential()
model.add(ll.InputLayer([img_size, img_size, 1]))

model.add(ll.Conv2D(filters=16, kernel_size=(3,3), strides=(1, 1), padding='same'))
model.add(keras.layers.LeakyReLU(alpha=alpha))
model.add(ll.Conv2D(filters=16, kernel_size=(3,3), strides=(1, 1), padding='same'))
model.add(keras.layers.LeakyReLU(alpha=alpha))
model.add(ll.MaxPool2D(pool_size=(pool_size,pool_size)))

model.add(ll.Dropout(dropout))

model.add(ll.Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), padding='same'))
model.add(keras.layers.LeakyReLU(alpha=alpha))
model.add(ll.Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), padding='same'))
model.add(keras.layers.LeakyReLU(alpha=alpha))
model.add(ll.MaxPool2D(pool_size=(pool_size,pool_size)))

model.add(ll.Dropout(dropout))

model.add(ll.Conv2D(filters=64, kernel_size=(3,3), strides=(1, 1), padding='same'))
model.add(keras.layers.LeakyReLU(alpha=alpha))
model.add(ll.Conv2D(filters=128, kernel_size=(3,3), strides=(1, 1), padding='same'))
model.add(keras.layers.LeakyReLU(alpha=alpha))
model.add(ll.MaxPool2D(pool_size=(pool_size,pool_size)))

model.add(ll.Flatten())

# model.add(ll.Dense(256, activation='elu', kernel_regularizer=regularizers.l2(l2_regularization)))
model.add(ll.Dense(256, activation='tanh', kernel_regularizer=regularizers.l2(l2_regularization)))
model.add(ll.Dense(9, activation="softmax"))

model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 25, 25, 1)         0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 25, 25, 16)        160       
_________________________________________________________________
leaky_re_lu_99 (LeakyReLU)   (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 25, 25, 16)        2320      
_________________________________________________________________
leaky_re_lu_100 (LeakyReLU)  (None, 25, 25, 16)        0         
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 12, 12, 16)        0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 12, 12, 16)        0         
__________

In [0]:
#@title Adjust training parameters { vertical-output: true, form-width: "10%", display-mode: "form" }
epochs = 50 #@param {type:"number"}
train_test_split = 0.8 #@param {type:"slider", min:0, max:1, step:0.05}

In [0]:
labels_new = to_categorical(train_labels)
model.fit(train[:int(train_test_split*train.shape[0])], labels_new[:int(train_test_split*train.shape[0])], validation_data = [train[int(train_test_split*train.shape[0]):], labels_new[int(train_test_split*train.shape[0]):]], epochs=epochs)

Train on 10204 samples, validate on 2552 samples
Epoch 1/50
10204/10204 [==============================] - 6s 596us/step - loss: 0.2817 - acc: 0.9369 - val_loss: 0.3916 - val_acc: 0.9193
Epoch 2/50
10204/10204 [==============================] - 6s 598us/step - loss: 0.2980 - acc: 0.9338 - val_loss: 0.3726 - val_acc: 0.9220
Epoch 3/50
10204/10204 [==============================] - 6s 604us/step - loss: 0.2874 - acc: 0.9366 - val_loss: 0.3957 - val_acc: 0.9220
Epoch 4/50
 1568/10204 [===>..........................] - ETA: 4s - loss: 0.3063 - acc: 0.931110204/10204 [==============================] - 6s 600us/step - loss: 0.2814 - acc: 0.9375 - val_loss: 0.3749 - val_acc: 0.9185
Epoch 5/50
10204/10204 [==============================] - 6s 602us/step - loss: 0.2797 - acc: 0.9366 - val_loss: 0.4004 - val_acc: 0.9228
Epoch 6/50
10204/10204 [==============================] - 6s 602us/step - loss: 0.2706 - acc: 0.9404 - val_loss: 0.3924 - val_acc: 0.9271
Epoch 7/50
 5760/10204 [===============>

In [0]:
with open('./models/model%0.2f.json' % (100*model.evaluate(train[int(0.8*train.shape[0]):], labels_new[int(0.8*train.shape[0]):])[1]), 'w') as model_json:
    model_json.write(model.to_json())
model.save_weights('./models/weights%0.2f.h5' % (100*model.evaluate(train[int(0.8*train.shape[0]):], labels_new[int(0.8*train.shape[0]):])[1]))

2552/2552 [==============================] - 0s 160us/step
